In [2]:
import re
import json
import jwt
import requests as r
from time import time, sleep
from cachetools import cached, TTLCache


with open("key.json") as f:
    settings = json.load(f)
service_account_id = settings["service_account_id"]
key_id = settings["id"]
private_key = settings["private_key"]
folder_id = "b1ggivrnbg1ftsr8no1s"


@cached(cache=TTLCache(maxsize=1024, ttl=3600))
def get_ya_token():
    now = int(time())
    payload = {
        "aud": "https://iam.api.cloud.yandex.net/iam/v1/tokens",
        "iss": service_account_id,
        "iat": now,
        "exp": now + 360,
    }
    
    encoded_token = jwt.encode(
        payload, private_key, algorithm="PS256", headers={"kid": key_id}
    )
    
    iam_token = r.post(
        "https://iam.api.cloud.yandex.net/iam/v1/tokens", json={"jwt": encoded_token}
    )
    if iam_token.status_code != 200:
        raise Exception("Wrong IAM token response")
    print(f"New IAM token, expires at {iam_token.json()['expiresAt']}")
    return iam_token.json()["iamToken"]


if __name__ == "__main__": 
    values = {
      "modelUri": f"gpt://{folder_id}/yandexgpt-lite/latest",
      "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": "2000"
      },
      "messages": [
        {
          "role": "system",
          "text": "Найди ошибки в тексте и исправь их"
        },
        {
          "role": "user",
          "text": "Ламинат подойдет для укладке на кухне или в детской комнате – он не боиться влаги и механических повреждений благодаря защитному слою из облицованных меламиновых пленок толщиной 0,2 мм и обработанным воском замкам."
        }
      ]
    }
    resp = r.post(
        f"https://llm.api.cloud.yandex.net/foundationModels/v1/completion", 
        json=values, 
        headers={"Authorization": f"Bearer {get_ya_token()}", "x-folder-id": folder_id}
    )
    display(resp.json())

New IAM token, expires at 2024-06-11T01:34:03.136548859Z


{'result': {'alternatives': [{'message': {'role': 'assistant',
     'text': '**Вот исправленный текст:**\n\nЛаминат подойдёт для укладки на кухне или в детской комнате — он не боится влаги и механических повреждений  благодаря защитному слою из **меламиновых** плёнок толщиной 0,2 мм и **специальной обработке** замков.'},
    'status': 'ALTERNATIVE_STATUS_FINAL'}],
  'usage': {'inputTextTokens': '70',
   'completionTokens': '55',
   'totalTokens': '125'},
  'modelVersion': '18.01.2024'}}